In [1]:
import numpy as np
from subprocess import check_output
import tempfile,os,sys
from collections import Counter
from HTKRW import openx
import theano
import theano.tensor as T
import lasagne
import struct
#**********************************
#       paramters                 *
#**********************************
PATH="/home/jango/distillation"
DIC=PATH+"/shared_babel/phone.dic"
LM=PATH+"/shared_babel/pdnet"
HLIST=PATH+"/shared_babel/monophones1"
SALIGN='-salign -sb 160 -b2 60 -s 1000 -m 4000 -quiet -1pass'
OPT="-sb 160 -b2 60 -s 1000 -m 4000 -quiet -1pass"
JULIUS='julius-4.3.1'
NUM_STATES=120
N_EPOCHS = 100
TIMESTEP=30
_EPSILON=1e-15
np.random.seed(55)
MODEL='teacher'
NET='IBO'
if MODEL=="teacher":
    N_INPUT=87
else:
    N_INPUT=39
#**********************************
#       HTK function              *
#**********************************
def write_htk(features,outputFileName,fs=100,dt=9):
    sampPeriod = 1./fs
    pk =dt & 0x3f
    features=np.atleast_2d(features)
    if pk==0:
        features =features.reshape(-1,1)
    with open(outputFileName,'wb') as fh:
        fh.write(struct.pack(">IIHH",len(features),sampPeriod*1e7,features.shape[1]*4,dt))
        features=features.astype(">f")
        features.tofile(fh)
#**********************************
#       julius recognizer         *
#**********************************
def RecogWithStateProbs(typ,s=2,pl=2):
    MODEL=PATH+"/%s/HMM/hmmdefs" %typ
    testlist=PATH+"/%s/feature/list/testdnn.list" %typ

    cmd='echo | %s -filelist %s -hlist %s -h %s -nlr %s -v %s %s -lmp %s %s %s' %(JULIUS,testlist, HLIST, MODEL, LM, DIC, OPT, s, pl,"-input outprob")
    result=check_output(cmd,shell=1).split("\n")
    #print cmd
    
    phone=["#!MLF!#\n"]
    f=open(testlist,"r")
    train=f.readlines()
    f.close()
    i=0
    #take result lines
    setname=testlist.split("/")[-1].split("mfc")[0]
    for r in result:
        if 'sentence1' in r:
            fn='"*/'+train[i].split("/")[-1][:-5]+'.rec"\n'
            rec=(("s_s"+r.split("<s>")[1]).replace("</s>","s_e")).replace(" ","\n")+"\n.\n"
            phone.append(fn+rec)
            i+=1
    #write mlf
    fw=open(PATH+"/%s/LSTMRec/rec.mlf" %typ,"w")
    for p in phone:
        fw.write(p)
    fw.close()
    #run HTK HResults
    cmd='HResults -A -z ::: -I '+PATH+'/shared_babel/mlf/alignedtest.mlf -e ::: s_s -e ::: s_e %s %s/%s/LSTMRec/rec.mlf' %(HLIST,PATH,typ)
    acc=check_output(cmd,shell=1)
    #print acc
    PER=100-float(acc.split("\n")[-3].split(" ")[2].split("=")[1])
    #print "%s    s: %s, p: %s, PER: %s" %(typ,s,pl,PER)
    return PER
#**********************************
#       data processing function  *
#**********************************
def per_sentence(seq_X,seq_Y,time_step=TIMESTEP):
    """lstm data generator 
    Args:
      seq_X: a sequence of vectors for one sentence (len, input_Dim)
      seq_Y: a sequence of vectors for one sentence (len, output_Dim)
      time_step: the length of history
    Returns:
      lstm_X: (n_size, timestep, input_Dim)
      lstm_Y: (n_size, )
      mask: for implementing variable length lstm
    note:
      
    """
    N_input=seq_X.shape[1]
    sen_len=len(seq_X)
    
    lstm_X = np.zeros((sen_len, time_step, N_input))
    lstm_M = np.ones((sen_len, time_step))

    for i in range(0,sen_len):
        if i<time_step-1:
            lstm_X[i]=np.vstack((seq_X[0:i+1],np.zeros((time_step-i-1,N_input))))
            lstm_M[i]=np.hstack((np.ones(i+1),np.zeros(time_step-i-1)))
        else:
            lstm_X[i]=seq_X[i-time_step+1:i+1]           
            lstm_M[i]=np.ones(time_step)
    return lstm_X.astype("float32"),seq_Y.astype('int16'),lstm_M.astype("int16")
def per_sentence2_noY(seq_X,time_step=TIMESTEP):
    """lstm data generator without Y
    Args:
      seq_X: a sequence of vectors for one sentence (len, input_Dim)
      time_step: the length of history
    Returns:
      lstm_X: (n_size, timestep, input_Dim)
      mask: for implementing variable length lstm
    note:
      
    """
    N_input=seq_X.shape[1]
    sen_len=len(seq_X)
    
    lstm_X = np.zeros((sen_len, time_step, N_input))
    lstm_M = np.ones((sen_len, time_step))

    for i in range(0,sen_len):
        if i<time_step-1:
            lstm_X[i]=np.vstack((seq_X[0:i+1],np.zeros((time_step-i-1,N_input))))
            lstm_M[i]=np.hstack((np.ones(i+1),np.zeros(time_step-i-1)))
        else:
            lstm_X[i]=seq_X[i-time_step+1:i+1]           
            lstm_M[i]=np.ones(time_step)
    return lstm_X.astype("float32"),lstm_M.astype("int16")
#--------------------------------------------------------------------------------------
def total_sentences(X,Y,time_step=TIMESTEP):
    """lstm data generator 
    return:
      total sentences tensors
    """
    XX=list()
    YY=list()
    MM=list()
    for x,y in zip(X,Y):
        xx,yy,mm=per_sentence(x,y,time_step)
        XX.append(xx)
        YY.append(yy)
        MM.append(mm)
    return np.concatenate(XX).astype("float32"),lasagne.utils.one_hot(np.concatenate(YY)).eval().astype("int16"),np.concatenate(MM).astype("int16")
#**********************************
#       read data                 *
#**********************************
X_TRAIN=np.load(PATH+"/"+MODEL+"/LSTMFile/%s_train_lstm.npy" %MODEL[:3])
Y_TRAIN=np.load(PATH+"/"+MODEL+"/LSTMFile/%s_train_target_lstm.npy" %MODEL[:3])
X_TRAIN,Y_TRAIN,M_TRAIN=total_sentences(X_TRAIN,Y_TRAIN,time_step=TIMESTEP)
print '\ntrain data'
print X_TRAIN.shape,Y_TRAIN.shape,M_TRAIN.shape
#read validation data
X_VALI=np.load(PATH+"/"+MODEL+"/LSTMFile/%s_validation_lstm.npy" %MODEL[:3])
Y_VALI=np.load(PATH+"/"+MODEL+"/LSTMFile/%s_validation_target_lstm.npy" %MODEL[:3])
X_VALI,Y_VALI,M_VALI=total_sentences(X_VALI,Y_VALI,time_step=TIMESTEP)
print 'vali data'
print X_VALI.shape,Y_VALI.shape,M_VALI.shape
#**********************************
#       evaluation functions      *
#**********************************
def get_acc_loss(get_out):
    real=T.matrix('real')
    pred=T.matrix('prediction')
    pred=T.clip(pred, _EPSILON, 1.0 - _EPSILON)
    vali_acc = theano.function([pred,real], T.mean(lasagne.objectives.categorical_accuracy(pred, real)))
    vali_loss = theano.function([pred,real], T.mean(lasagne.objectives.categorical_crossentropy(pred, real)))
    P=all_predictions(get_out)
    return vali_acc(P,Y_VALI),vali_loss(P,Y_VALI)
#--------------------------------------------------------------------------------------
def all_predictions(get_out,batch=256):
    all_pred=list()
    i=0
    while True:
        pred=get_out(X_VALI[i:i+batch],M_VALI[i:i+batch])
        i=i+batch
        all_pred.append(pred)
        if i >= X_VALI.shape[0]:
            return np.concatenate(all_pred)
#--------------------------------------------------------------------------------------
def display(fname,acc,loss,accvali,lossvali,b,n_samples,times,PER):
        sys.stdout.write('Epoch:%2.2s(%4.4s) | Best acc:%5.5s loss:%5.5s | Cur acc:%5.5s loss:%5.5s | PER:%5.5s\r' %(times,round((float(b)/n_samples)*100,1),round(accvali,3),round(lossvali,3),round(acc,3),round(loss,3),round(PER,3)))
        sys.stdout.flush()
#--------------------------------------------------------------------------------------
def writer(fname,acc,loss,accvali,lossvali,b,n_samples,times,PER):
    f=open(fname[:-4]+'.txt','a')
    f.write('Epoch:%2.2s | Best acc:%5.5s loss:%5.5s | Cur acc:%5.5s loss:%5.5s | PER:%5.5s\n' %(times,round(accvali,3),round(lossvali,3),round(acc,3),round(loss,3),round(PER,3)))
    f.close()
def prediction(X,get_out):
    X,M=per_sentence2_noY(X,time_step=TIMESTEP)
    pred=get_out(X,M)
    return pred
#**********************************
#   get results                   *
#**********************************
def GetStateProbs(typ,n_hidden, n_cell, net, drp, B,get_out):
    #load state priors
    priors=dict([line.split() for line in open(PATH+"/%s/StatPrior_train" %typ).readlines()])
    #change datatype
    priors=dict([(int(a),float(b)) for a, b in priors.items()])
    #turn into 1D vector in order
    priors=np.array([priors[i] for i in range(len(priors))])
    #get file list
    fnames=open(PATH+"/%s/feature/list/testmfc.list" %typ).readlines()
    #
    #loop all files in filelist
    for name in fnames:
        #print name[:-1]
        fmfc=openx(name[:-1],'rb',veclen=87)
        data=fmfc.getall()
        #make windows
        X=data.astype('float32')
        #get result from DNN
        probs=prediction(X,get_out)
        #turn into likelihoods
        log_liks=np.log10(probs/priors)
        #print log_liks.shape
        prob_path=PATH+"/%s/StatePro/" %typ+name.split("/")[-1][:-4]+"llk"
        #print prob_path
        write_htk(log_liks,prob_path)
    PER=RecogWithStateProbs(typ,2,2)
    return PER
#**********************************
#   network builder               *
#**********************************
def Dense(l_in, n_hidden, in_reshape, out_reshape, drp, init_b=0.001, timestep=TIMESTEP,n_features=N_INPUT):
    """Dense layer creater
    Args:
      l_in: incoming layer
      n_hidden: hidden layer nodes
      in_reshape: whether to reshape for FF
      in_reshape: whether to reshape for LSTM
      drp: drop out probability
      init_b: init bias for FF
      timestep: length of history to look
      n_features: input vector dimenstion
    Return:
      instance of lasagne layer
    """
    #reshape for feedforward
    if in_reshape == True:
        l_in = lasagne.layers.ReshapeLayer(l_in, (-1, n_features))
        print 'Reshape for FF'
    #build dense layer
    l_in = lasagne.layers.DenseLayer(
        l_in,
        n_hidden, 
        b=lasagne.init.Constant(init_b),
        nonlinearity=lasagne.nonlinearities.rectify)
    print 'Add Dense'
    print 'Add Dropout'
    #reshape for blstm
    if out_reshape == True:
        l_in = lasagne.layers.ReshapeLayer(l_in, (-1,timestep, n_hidden))
        print 'Reshape for BLSTM'
    #add dropout layer
    layer=lasagne.layers.dropout(l_in,drp)
    return layer
#--------------------------------------------------------------------------------------
def BLSTM(l_in, l_mask, n_cell, last, drp, f_bias=1.0):
    """BLSTM layer creater
    Args:
      l_in: incoming layer
      l_mask: lasagne mask layer
      n_cell: size of lstm cell
      last: whether to only return final sequential output
      drp: drop out probability
      f_bias: init bias for forget gate
    Return:
      instance of lasagne layer
    """
    #parameters of LSTM
    gate_parameters = lasagne.layers.recurrent.Gate(
        W_in=lasagne.init.Orthogonal(), W_hid=lasagne.init.Orthogonal(),
        b=lasagne.init.Constant(0.))
    forget_gate = lasagne.layers.recurrent.Gate(
        W_in=lasagne.init.Orthogonal(), W_hid=lasagne.init.Orthogonal(),
        b=lasagne.init.Constant(1.0))
    cell_parameters = lasagne.layers.recurrent.Gate(
        W_in=lasagne.init.Orthogonal(), W_hid=lasagne.init.Orthogonal(),
        W_cell=None, b=lasagne.init.Constant(0.),
        nonlinearity=lasagne.nonlinearities.tanh)
    #BLSTM
    l_lstm = lasagne.layers.recurrent.LSTMLayer(
        l_in, n_cell,
        mask_input=l_mask,
        ingate=gate_parameters, forgetgate=gate_parameters,
        cell=cell_parameters, outgate=forget_gate,
        learn_init=True, grad_clipping=100., backwards=False, only_return_final=last)
    l_lstm_b = lasagne.layers.recurrent.LSTMLayer(
        l_in, n_cell,
        mask_input=l_mask,
        ingate=gate_parameters, forgetgate=gate_parameters,
        cell=cell_parameters, outgate=forget_gate,
        learn_init=True, grad_clipping=100., backwards=True, only_return_final=last)
    if last==True:
        l_in = lasagne.layers.ConcatLayer([l_lstm, l_lstm_b])
        print 'Add last BLSTM'
    else:
        l_in = lasagne.layers.ElemwiseSumLayer([l_lstm, l_lstm_b])
        print 'Add BLSTM'
    layer=lasagne.layers.dropout(l_in,drp)
    print 'Add Dropout'
    return layer
#--------------------------------------------------------------------------------------
def Network(l_in, l_mask, n_class, n_hidden, n_cell, net, drp, n_input=N_INPUT):
    """network builder
    Args:
      l_in: lasagne input layer
      l_mask: lasagne mask layer
      n_class: number of labels
      n_hidden: number of hidden layer nodes
      n_cell: size of lstm cell
      net: a string denoting the type of network, starting with 'I', ending with 'O'.
          I: input layer
          F: feedforward layer
          B: blstm layer
          O: output layer
          note: the net can contain no dense layer, but must contain at least one blstm
      drp: drop out probability
      n_input: input vector dimenstion
    Return:
      instance of lasagne layer
    """
    print 'Add Inputlayer'
    #hidden layers
    for i,l in enumerate(net):
        #add dense net
        if l == 'F':
            #if it's the first dense_layer, reshape will be performed to use feedforward
            if i == 1:
                in_reshape=True
            else:
                in_reshape=False
            #if it's the last dense_layer, reshape will be performed to use blstm
            if net[i+1] == 'B':
                out_reshape=True
            else:
                out_reshape=False
            l_in=Dense(l_in, n_hidden, in_reshape, out_reshape, drp, init_b=0.001)
        #add blstm net
        elif l == 'B':
            #if it's the last blstm net, only return final sequential output
            if net[i+1] == 'B':
                last=False
            else:
                last=True
            l_in=BLSTM(l_in, l_mask, n_cell, last, drp, f_bias=1.0)
    #output layer       
    l_out = lasagne.layers.DenseLayer(l_in, num_units=n_class, nonlinearity=lasagne.nonlinearities.softmax)
    print 'Add output softmax'
    return l_out
#**********************************
#   train functions               *
#**********************************
def shuffle3lists(x,y,m):
    ri=np.random.permutation(len(x))
    return x[ri],y[ri],m[ri]
#--------------------------------------------------------------------------------------

Using gpu device 0: GeForce GTX 980 Ti (CNMeM is disabled, cuDNN 5004)
/usr/local/anaconda2/lib/python2.7/site-packages/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")



train data
(913358, 30, 87) (913358, 120) (913358, 30)
vali data
(73159, 30, 87) (73159, 120) (73159, 30)


In [19]:
l_in = lasagne.layers.InputLayer(shape=(None, None, 87))
l_mask = lasagne.layers.InputLayer(shape=(None, None))
targets = T.imatrix('targets')
    
    #build network and return output
l_out=Network(l_in, l_mask, 120, 10, 10, 'IBO', 0.5, n_input=N_INPUT)
    #params
all_params = lasagne.layers.get_all_params(l_out)
    #output with dropout for train
l_out_train=lasagne.layers.get_output(l_out, deterministic=False)
    #clip
l_out_train=T.clip(l_out_train, _EPSILON, 1.0 - _EPSILON)
    #output with dropout for eval
l_out_eval=lasagne.layers.get_output(l_out, deterministic=True)
    #clip
l_out_eval=T.clip(l_out_eval, _EPSILON, 1.0 - _EPSILON)
    #losses
loss_train = T.mean(lasagne.objectives.categorical_crossentropy(l_out_train, targets))
loss_eval = T.mean(lasagne.objectives.categorical_crossentropy(l_out_eval, targets))
    #update
updates = lasagne.updates.adam(loss_train, all_params,learning_rate=0.1)
    #train
train = theano.function([l_in.input_var, targets, l_mask.input_var], loss_train, updates=updates)
    #get output of network
get_out=theano.function([l_in.input_var, l_mask.input_var],l_out_eval)
    #train of network with acc and loss

acc_eval = T.mean(lasagne.objectives.categorical_accuracy(l_out_eval, targets))


Add Inputlayer
Add last BLSTM
Add Dropout
Add output softmax


In [20]:
train_acc_loss = theano.function([l_in.input_var, targets, l_mask.input_var], [loss_train,loss_eval,acc_eval], updates=updates)

In [94]:
_,t_loss,t_acc=train_acc_loss(X_TRAIN[:5],Y_TRAIN[:5],M_TRAIN[:5])

In [95]:
_

array(0.2180037413550963)